<a href="https://colab.research.google.com/github/SatriaImawan12/Top-Ranked-Gutenberg-Ebooks-Download/blob/main/Top_Ranked_Gutenberg_Ebooks_Download.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Top Gutenberg Ebooks (yesterday's ranking) download

### What is Project Gutenberg? -
Project Gutenberg is a volunteer effort to digitize and archive cultural works, to "encourage the creation and distribution of eBooks". It was founded in 1971 by American writer Michael S. Hart and is the **oldest digital library.** This longest-established ebook project releases books that entered the public domain, and can be freely read or downloaded in various electronic formats.

* **This starter code scrapes the url of the Project Gutenberg's Top 100 ebooks (yesterday's ranking) for identifying the ebook links.**
* **It uses BeautifulSoup4 for parsing the HTML and regular expression code for identifying the Top 100 ebook file numbers.**
* **It includes a function to take an usser input on how many books to download and then crawls the server to download them in a dictionary object.**
* **Finally, it also includes a function to save the downloaded Ebooks as text files in a local directory.**

In [ ]:
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import ssl
import re

#### Ignore SSL certificate errors

In [ ]:
# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

#### Read the HTML from the URL and pass on to BeautifulSoup

In [ ]:
# Read the HTML from the URL and pass on to BeautifulSoup
top100url = 'https://www.gutenberg.org/browse/scores/top'
url = top100url
print(f"Opening the file connection to {url}")
html = urllib.request.urlopen(url, context=ctx).read()
soup = BeautifulSoup(html, 'html.parser')
print("Connection established and HTML parsed...")

Opening the file connection to https://www.gutenberg.org/browse/scores/top
Connection established and HTML parsed...


#### Find all the _'href'_ tags and store them in the list of links

In [ ]:
# Empty list to hold all the http links in the HTML page
lst_links=[]

In [ ]:
# Find all the href tags and store them in the list of links
for link in soup.find_all('a'):
    #print(link.get('href'))
    lst_links.append(link.get('href'))

#### Use regular expression to find the numeric digits in these links. These are the file number for the Top 100 books.

In [ ]:
# Use regular expression to find the numeric digits in these links. These are the file number for the Top 100 books.
# Initialize empty list to hold the file numbers
booknum=[]

In [ ]:
# Number 19 to 119 in the original list of links have the Top 100 books' number.
for i in range(19,119):
	link=lst_links[i]
	link=link.strip()
	# Regular expression to find the numeric digits in the link (href) string
	n=re.findall('[0-9]+',link)
	if len(n)==1:
		# Append the filenumber casted as integer
		booknum.append(int(n[0]))

print ("\nThe file numbers for the top 100 ebooks on Gutenberg are shown below\n"+"-"*70)
print(booknum)


The file numbers for the top 100 ebooks on Gutenberg are shown below
----------------------------------------------------------------------
[1, 1, 7, 7, 30, 30, 1513, 2701, 1342, 145, 2641, 84, 100, 37106, 67979, 11, 16389, 6761, 394, 2160, 1259, 5197, 6593, 4085, 2554, 345, 5200, 2542, 28054, 174, 50150, 33283, 98, 2600, 76, 64317, 25344, 1661, 16119, 6130, 1998, 4300, 1400, 844, 2000, 16328, 30254, 1184, 74239, 2591, 27827, 1260, 74243, 74240, 1232, 4363, 1952, 8492, 1080, 24022, 5740, 768, 1727, 74244, 45, 74, 43, 219, 74245, 244, 67098, 31552, 2680, 36034, 16, 1497, 12, 2650, 398, 23, 8800, 3207, 408, 31284, 41, 996, 514, 120, 135, 5827, 36]


#### Search in the extracted text (using Regular Expression) from the soup object to find the names of top 100 Ebooks (Yesterday's rank)

In [ ]:
start_idx=soup.text.splitlines().index('Top 100 EBooks yesterday')
lst_titles_temp=[] # Empty list of Ebook names
for i in range(100):
    lst_titles_temp.append(soup.text.splitlines()[start_idx+2+i])

In [ ]:
# Use regular expression to extract only text from the name strings and append to an empty list
lst_titles=[]
for i in range(100):
    id1,id2=re.match('^[a-zA-Z ]*',lst_titles_temp[i]).span()
    lst_titles.append(lst_titles_temp[i][id1:id2])
for l in lst_titles:
    print(l)

Romeo and Juliet by William Shakespeare 
Moby Dick
Pride and Prejudice by Jane Austen 
Middlemarch by George Eliot 
A Room with a View by E
Frankenstein
The Complete Works of William Shakespeare by William Shakespeare 
Little Women
The Blue Castle
Alice
The Enchanted April by Elizabeth Von Arnim 
The Adventures of Ferdinand Count Fathom 
Cranford by Elizabeth Cleghorn Gaskell 
The Expedition of Humphry Clinker by T
Twenty years after by Alexandre Dumas and Auguste Maquet 
My Life 
History of Tom Jones
The Adventures of Roderick Random by T
Crime and Punishment by Fyodor Dostoyevsky 
Dracula by Bram Stoker 
Metamorphosis by Franz Kafka 
A Doll
The Brothers Karamazov by Fyodor Dostoyevsky 
The Picture of Dorian Gray by Oscar Wilde 
The Devil is an Ass by Ben Jonson 
Calculus Made Easy by Silvanus P
A Tale of Two Cities by Charles Dickens 
War and Peace by graf Leo Tolstoy 
Adventures of Huckleberry Finn by Mark Twain 
The Great Gatsby by F
The Scarlet Letter by Nathaniel Hawthorne 
The A

### Define a function that takes an user input of how many top books to download and crawls the server to download

In [ ]:
def download_top_books(num_download=10,verbosity=0):
    """
    Function: Download top N books from Gutenberg.org where N is specified by user
    Verbosity: If verbosity is turned on (set to 1) then prints the downloading status for every book
    Returns: Returns a dictionary where keys are the names of the books and values are the raw text.
    Exception Handling: If a book is not found on the server (due to broken link or whatever reason), inserts "NOT FOUND" as the text.
    """
    topEBooks = {}

    if num_download<=0:
        print("I guess no download is necessary")
        return topEBooks

    if num_download>100:
        print("You asked for more than 100 downloads.\nUnfortunately, Gutenberg ranks only top 100 books.\nProceeding to download top 100 books.")
        num_download=100

    # Base URL for files repository
    baseurl= 'http://www.gutenberg.org/files/'

    if verbosity==1:
        count_done=0
        for i in range(num_download):
            print ("Working on book:", lst_titles[i])

            # Create the proper download link (url) from the book id
            # You have to examine the Gutenberg.org file structure carefully to come up with the proper url
            bookid=booknum[i]
            bookurl= baseurl+str(bookid)+'/'+str(bookid)+'-0.txt'
            # Create a file handler object
            try:
                fhand = urllib.request.urlopen(bookurl)
                txt_dump = ''
                # Iterate over the lines in the file handler object and dump the data into the text string
                for line in fhand:
                    # Use decode method to convert the UTF-8 to Unicode string
                    txt_dump+=line.decode()
                # Add downloaded text to the dictionary with keys matching the list of book titles.
                # This puts the raw text as the value of the key of the dictionary bearing the name of the Ebook
                topEBooks[lst_titles[i]]=txt_dump
                count_done+=1
                print (f"Finished downloading {round(100*count_done/num_download,2)}%")
            except urllib.error.URLError as e:
                topEBooks[lst_titles[i]]="NOT FOUND"
                count_done+=1
                print(f"**ERROR: {lst_titles[i]} {e.reason}**")
    else:
        count_done=0
        from tqdm import tqdm, tqdm_notebook
        for i in tqdm(range(num_download),desc='Download % completed',dynamic_ncols=True):
            # Create the proper download link (url) from the book id
            # You have to examine the Gutenberg.org file structure carefully to come up with the proper url
            bookid=booknum[i]
            bookurl= baseurl+str(bookid)+'/'+str(bookid)+'-0.txt'
            # Create a file handler object
            try:
                fhand = urllib.request.urlopen(bookurl)
                txt_dump = ''
                # Iterate over the lines in the file handler object and dump the data into the text string
                for line in fhand:
                    # Use decode method to convert the UTF-8 to Unicode string
                    txt_dump+=line.decode()
                # Add downloaded text to the dictionary with keys matching the list of book titles.
                # This puts the raw text as the value of the key of the dictionary bearing the name of the Ebook
                topEBooks[lst_titles[i]]=txt_dump
                count_done+=1
            except urllib.error.URLError as e:
                topEBooks[lst_titles[i]]="NOT FOUND"
                count_done+=1
                print(f"**ERROR: {lst_titles[i]} {e.reason}**")

    print ("-"*40+"\nFinished downloading all books!\n"+"-"*40)

    return (topEBooks)

#### Test the function with verbosity=0 (default)

In [ ]:
dict_books=download_top_books(1)

Download % completed: 100%|██████████| 1/1 [00:00<00:00,  1.07it/s]

----------------------------------------
Finished downloading all books!
----------------------------------------


#### Show the final dictionary and an example of the downloaded text

In [ ]:
print(dict_books[lst_titles[0]][:1500])




     NOTE:  This file combines the first two Project Gutenberg
     files, both of which were given the filenumber #1. There are
     several duplicate files here. There were many updates over
     the years.  All of the original files are included in the
     "old" subdirectory which may be accessed under the "More
     Files" listing in the PG Catalog of this file. No changes
     have been made in these original etexts.



**Welcome To The World of Free Plain Vanilla Electronic Texts**

**Etexts Readable By Both Humans and By Computers, Since 1971**

*These Etexts Prepared By Hundreds of Volunteers and Donations*

Below you will find the first nine Project Gutenberg Etexts, in
one file, with one header for the entire file.  This is to keep
the overhead down, and in response to requests from Gopher site
keeper to eliminate as much of the headers as possible.

However, for legal and financial reasons, we must request these
headers be left at the beginning of each file that is poste

### Write a function to download and save the downloaded texts

In [ ]:
def save_text_files(num_download=10,verbosity=1):
    """
    Downloads top N books from Gutenberg.org where N is specified by user.
    If verbosity is turned on (set to 1) then prints the downloading status for every book.
    Asks user for a location on computer where to save the downloaded Ebooks and process accordingly.
    Returns status message indicating how many ebooks could be successfully downloaded and saved
    """

    import os

    # Download the Ebooks and save in a dictionary object (in-memory)
    dict_books=download_top_books(num_download=num_download,verbosity=verbosity)

    if dict_books=={}:
        return None

    # Ask use for a save location (directory path)
    savelocation=input("Please enter a folder location to save the Ebooks in:")

    count_successful_download=0

    # Create a default folder/directory in the current working directory if the input is blank
    if (len(savelocation)<1):
        savelocation=os.getcwd()+'\\'+'Ebooks'+'\\'
        # Creates new directory if the directory does not exist. Otherwise, just use the existing path.
        if not os.path.isdir(savelocation):
            os.mkdir(savelocation)
    else:
        if savelocation[-1]=='\\':
            os.mkdir(savelocation)
        else:
            os.mkdir(savelocation+'\\')
    #print("Saving files at:",savelocation)
    for k,v in dict_books.items():
        if (v!="NOT FOUND"):
            filename=savelocation+str(k)+'.txt'
            file=open(filename,'wb')
            file.write(v.encode("UTF-8",'ignore'))
            file.close()
            count_successful_download+=1

    # Status message
    print (f"{count_successful_download} book(s) was/were successfully downloaded and saved to the location {savelocation}")
    if (num_download!=count_successful_download):
        print(f"{num_download-count_successful_download} books were not found on the server!")

In [ ]:
save_text_files(90,verbosity=1)

Working on book: Romeo and Juliet by William Shakespeare 
Finished downloading 1.11%
Working on book: Moby Dick
Finished downloading 2.22%
Working on book: Pride and Prejudice by Jane Austen 
Finished downloading 3.33%
Working on book: Middlemarch by George Eliot 
Finished downloading 4.44%
Working on book: A Room with a View by E
Finished downloading 5.56%
Working on book: Frankenstein
Finished downloading 6.67%
Working on book: The Complete Works of William Shakespeare by William Shakespeare 
Finished downloading 7.78%
Working on book: Little Women
Finished downloading 8.89%
Working on book: The Blue Castle
Finished downloading 10.0%
Working on book: Alice
Finished downloading 11.11%
Working on book: The Enchanted April by Elizabeth Von Arnim 
Finished downloading 12.22%
Working on book: The Adventures of Ferdinand Count Fathom 
Finished downloading 13.33%
Working on book: Cranford by Elizabeth Cleghorn Gaskell 
Finished downloading 14.44%
Working on book: The Expedition of Humphry C